In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
NUM_DATA_POINTS = 100000

# Introduction

I just finished studying Brady Neal's [Introduction to Causal Inference](https://www.bradyneal.com/causal-inference-course) course. It was a great introduction to the topic for me, and the author was kind enough to release both the lectures and the associated course book for free (at the time of writing this there were a few chapters not yet published). The main goal of this article is to get my hands dirty and solidify some of the things I learned - even better if it turns out helping someone else as well.

Let's start off with some semantics. The following figure from the course book illustrates a general procedure of causal inference - the flow from a causal estimand + causal model to a statistical estimand, and from a statistical estimand + data to an estimate. Here is a short summary of what the different terms mean:

* Causal Estimand
  * The causal expression we are interested in evaluating.
  * Contains either one or more potential outcomes, e.g. $Y(t)$, or one ore more do-operators, e.g. $P(Y|do(T=t))$.
  * In this article I will primarily consider the causal estimand *average treatment effect* (ATE): $E[Y(1) - Y(0)] = E[Y|do(T=1)] - E[Y|do(T=0)]$
* Statistical Estimand:
  * Identification etc
* Estimate:
  * Estimation
* Causal Model:
  * Used to identify the causal estimand
  * DAG vs SCM
* Data:
  * Used in estimation of statistical estimand to get estimate

![figures/identification_estimation_flowchart.png](figures/identification_estimation_flowchart.png)

Conditional Outcome Modeling (COM) estimation.

# Causal Model

## Graphical Model (DAG)

![figures/dag_1.png](figures/dag_1.png)

## Structural Causal Model (SCM)

## Data generating process

In [2]:
def f_w():
    u_w = np.random.choice([0, 1])
    return u_w

def f_t(w):
    u_t = np.random.normal()
    intermediate = 0.5 * w + u_t
    return 1 if intermediate > 0 else 0

def f_y(w, t):
    u_y = np.random.normal()
    return 0.8 * w + 1.2 * t + u_y

W = np.array([f_w() for _ in range(NUM_DATA_POINTS)])
T = np.array([f_t(w) for w in W])
Y = np.array([f_y(w, t) for w, t in zip(W, T)])

# Identification

Backdoor criterion

Backdoor adjustment

# Estimation

Linear model

In [3]:
WT = np.array([W, T]).T
WT_1 = np.array([W, np.ones(len(W))]).T
WT_0 = np.array([W, np.zeros(len(W))]).T
model = LinearRegression()
model.fit(WT, Y)
ate_estimate = np.mean(model.predict(WT_1) - model.predict(WT_0))
print("ATE estimate:", ate_estimate)

ATE estimate: 1.2012474981998513


In [4]:
print("ATE estimate:", model.coef_[1])

ATE estimate: 1.2012474981998507


# Colliders

![figures/dag_2.png](figures/dag_2.png)

In [5]:
def f_w():
    u_w = np.random.choice([0, 1])
    return u_w

def f_t(w):
    u_t = np.random.normal()
    intermediate = 0.5 * w + u_t
    return 1 if intermediate > 0 else 0

def f_y(w, t):
    u_y = np.random.normal()
    return 0.8 * w + 1.2 * t + u_y

def f_z(t, y):
    u_z = np.random.normal()
    intermediate = 1.5 * t + y - 2 + u_z
    return 1 if intermediate > 0 else 0

W = np.array([f_w() for _ in range(NUM_DATA_POINTS)])
T = np.array([f_t(w) for w in W])
Y = np.array([f_y(w, t) for w, t in zip(W, T)])
Z = np.array([f_z(t, y) for t, y in zip(T, Y)])

In [6]:
WZT = np.array([W, Z, T]).T
WZT_1 = np.array([W, Z, np.ones(len(W))]).T
WZT_0 = np.array([W, Z, np.zeros(len(W))]).T
model = LinearRegression()
model.fit(WZT, Y)
ate_estimate = np.mean(model.predict(WZT_1) - model.predict(WZT_0))
print("ATE estimate:", ate_estimate)

ATE estimate: 0.40757848747942743


In [7]:
WT = np.array([W, T]).T
WT_1 = np.array([W, np.ones(len(W))]).T
WT_0 = np.array([W, np.zeros(len(W))]).T
model = LinearRegression()
model.fit(WT, Y)
ate_estimate = np.mean(model.predict(WT_1) - model.predict(WT_0))
print("ATE estimate:", ate_estimate)

ATE estimate: 1.196696813274864


# Unobserved confounding

![figures/dag_3.png](figures/dag_3.png)

In [8]:
def f_w():
    u_w = np.random.choice([0, 1])
    return u_w

def f_u():
    u_u = np.random.normal()
    return u_u

def f_t(w, u):
    u_t = np.random.normal()
    intermediate = 0.5 * w + 0.8 * u + u_t
    return 1 if intermediate > 0 else 0

def f_y(w, u, t):
    u_y = np.random.normal()
    return 0.8 * w - 2 * u + 1.2 * t + u_y

W = np.array([f_w() for _ in range(NUM_DATA_POINTS)])
U = np.array([f_u() for _ in range(NUM_DATA_POINTS)])
T = np.array([f_t(w, u) for w, u in zip(W, U)])
Y = np.array([f_y(w, u, t) for w, u, t in zip(W, U, T)])

In [9]:
WT = np.array([W, T]).T
WT_1 = np.array([W, np.ones(len(W))]).T
WT_0 = np.array([W, np.zeros(len(W))]).T
model = LinearRegression()
model.fit(WT, Y)
ate_estimate = np.mean(model.predict(WT_1) - model.predict(WT_0))
print("ATE estimate:", ate_estimate)

ATE estimate: -0.8108111630460578


## Sensitivity analysis

# Conclusion

TODO

Just a small part of causal inference